# Load libraries and data

Enter your project root:

In [1]:
project_root = '/home/dan1dr/zrive-ds-4q24-churn'

In [2]:
import sys
import os
import pandas as pd
import numpy as np
import configparser


# Define the project root path
current_wd = os.getcwd()

# Change the working directory if necessary
if current_wd != project_root:
    print(f"Changing working directory from {current_wd} to {project_root}")
    os.chdir(project_root)
else:
    print("Already in the correct path")

# Add 'src' directory to sys.path
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    print(f"Adding {src_path} to sys.path")
    sys.path.insert(0, src_path)

# Import the modules
from db_connectors.bigquery_service import BigqueryService
from data_gathering import data_gathering
from utils.logger import get_logger
logger = get_logger(__name__)

Changing working directory from /home/dan1dr/zrive-ds-4q24-churn/src/eda to /home/dan1dr/zrive-ds-4q24-churn
Adding /home/dan1dr/zrive-ds-4q24-churn/src to sys.path


### Extract the data from BigQuery

Define the query:

In [3]:
query_sql_22 = """WITH selectable_customer as (
  SELECT customer_id
  FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
  WHERE IS_CUST_SEGM_RESI > 0 
  AND IS_CUST_BILL_POST_CURR = TRUE
  AND CUST_BUNDLE_CURR = 'FMC'
  AND NUM_IMPAGOS = 0
  AND pago_final_0 IS NOT NULL
  GROUP BY customer_id
),

customer_selected as (
  SELECT customer_id as selected_customer
  FROM selectable_customer
  WHERE RAND() < 0.1
)

SELECT 
customer_id, MONTH, YEAR, pago_final_0, dif_pago_final_prev_month, dif_pago_final_prev_2_month, dif_pago_final_prev_3_month, periodica_0, dif_periodica_prev_month, dif_periodica_prev_2_month, 
dif_periodica_prev_3_month, consumo_0, dif_consumo_prev_month, dif_consumo_prev_2_month, dif_consumo_prev_3_month, aperiodica_0, dif_aperiodica_prev_month, 
dif_aperiodica_prev_2_month, dif_aperiodica_prev_3_month, discount_0, dif_discount_prev_month, dif_discount_prev_2_month, dif_discount_prev_3_month, ajuste_0, 
dif_ajuste_prev_month, dif_ajuste_prev_2_month, dif_ajuste_prev_3_month, Tota_Compra_disp, Curr_Compra_disp, Curr_Compra_Finanz_disp, Curr_Finanz_disp, Month_purchase_disp, Modelo_disp, Import_Rest_quota_disp, pvp_total_disp, pvp_total_disp_movil, Curr_cancel_disp, Tota_cancel_disp
NUM_GB_OWNN_CURR, NUM_GB_2G_CURR, NUM_GB_3G_CURR, NUM_GB_4G_CURR, NUM_GB_5G_CURR, NUM_SESS_CURR, NUM_SECS_CURR, NUM_CALL_CURR, NUM_CALL_WEEK_CURR, NUM_CALL_WEEKEND_CURR, 
NUM_SECS_WEEK_CURR, NUM_SECS_WEEKEND_CURR, NUM_CALL_WEEK, NUM_CALL_WEEKEND, NUM_DAYS_LINE_TYPE_FIXE_POST_DEA
FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
INNER JOIN customer_selected
ON customer_id = selected_customer
WHERE IS_CUST_SEGM_RESI > 0 
AND IS_CUST_BILL_POST_CURR = TRUE
AND CUST_BUNDLE_CURR = 'FMC'
AND NUM_IMPAGOS = 0
AND pago_final_0 IS NOT NULL
  """

In [18]:
query_sql_23 = """SELECT *
FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
  """

In [5]:
sample = data_gathering(query_sql_22)
logger.info(f"Extraction completed - data_2022")
#data_2023 = data_gathering(query_sql_23)
#logging.info(f"Extraction completed - data_2023")

INFO - Extraction completed - data_2022


### Save the data

In [8]:
save_path = os.path.join(project_root, 'data')
sample.to_parquet(os.path.join(save_path, 'subsample_users.parquet'))

### Load the data

In [9]:
save_path = '/home/dan1dr/zrive-ds-4q24-churn/data'
read_path = os.path.join(save_path, 'subsample_users.parquet')
sample = pd.read_parquet(read_path)

## Explore data

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [9]:
sample.head()

,customer_id,MONTH,YEAR,pago_final_0,dif_pago_final_prev_month,dif_pago_final_prev_2_month,dif_pago_final_prev_3_month,periodica_0,dif_periodica_prev_month,dif_periodica_prev_2_month,dif_periodica_prev_3_month,consumo_0,dif_consumo_prev_month,dif_consumo_prev_2_month,dif_consumo_prev_3_month,aperiodica_0,dif_aperiodica_prev_month,dif_aperiodica_prev_2_month,dif_aperiodica_prev_3_month,discount_0,dif_discount_prev_month,dif_discount_prev_2_month,dif_discount_prev_3_month,ajuste_0,dif_ajuste_prev_month,dif_ajuste_prev_2_month,dif_ajuste_prev_3_month,Tota_Compra_disp,Curr_Compra_disp,Curr_Compra_Finanz_disp,Curr_Finanz_disp,Month_purchase_disp,Modelo_disp,Import_Rest_quota_disp,pvp_total_disp,pvp_total_disp_movil,Curr_cancel_disp,NUM_GB_OWNN_CURR,NUM_GB_2G_CURR,NUM_GB_3G_CURR,NUM_GB_4G_CURR,NUM_GB_5G_CURR,NUM_SESS_CURR,NUM_SECS_CURR,NUM_CALL_CURR,NUM_CALL_WEEK_CURR,NUM_CALL_WEEKEND_CURR,NUM_SECS_WEEK_CURR,NUM_SECS_WEEKEND_CURR,NUM_CALL_WEEK,NUM_CALL_WEEKEND,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA
0,4142337,12,2022,87.1499,-0.92,5.00,13.15,123.2162,0.0,0.82,2.23,440.7130,-83.33,109.00,-288.48,0.0,0.00,0.00,0.0,-476.7793,82.40,-104.83,299.39,0.0,0.0,0.0,0.0,6,0,0,2,3,1,758.676250,2355.36,2355.36,0,0,0.000379,1.478475,34.539025,8.552260,12871,88554,820,624,196,69564,18990,471,140,<NA>
1,620962,10,2022,88.0450,22.35,4.19,15.41,174.9015,0.0,-2.74,-6.01,244.8217,-165.29,-60.96,-95.95,0.0,23.85,0.00,0.0,-331.6782,163.79,67.89,117.37,0.0,0.0,0.0,0.0,9,0,0,2,9,1,173.550000,1433.05,1433.05,0,0,0.003215,0.943611,14.486598,0.000000,11221,104166,564,408,156,68144,36022,300,113,<NA>
2,127593,01,2022,177.5206,0.89,22.48,2.95,254.5857,0.0,-1.35,-5.09,69.6490,33.88,29.52,7.54,0.0,0.00,-0.73,0.0,-146.7141,-32.99,-4.96,0.50,0.0,0.0,0.0,0.0,8,0,0,0,18,1,54.546250,1436.23,1436.23,0,0,0.000995,0.084578,24.819340,0.000000,4390,9911,154,109,45,7230,2681,100,34,<NA>
3,1464466,07,2022,106.6281,7.58,7.51,11.08,218.4490,-2.3,-2.30,-1.69,123.9285,-39.25,-67.05,-155.04,0.0,0.00,0.00,0.0,-235.7494,49.13,76.86,167.81,0.0,0.0,0.0,0.0,6,0,0,1,3,1,212.394583,1572.89,1434.05,0,0,0.014278,2.380983,23.064308,1.185673,15480,34602,476,337,139,23975,10627,308,115,<NA>
4,311788,11,2022,159.0900,-0.60,-1.50,3.00,217.1160,0.0,0.00,0.52,125.9174,-29.83,-58.43,-20.02,0.0,0.00,0.00,0.0,-183.9434,29.23,56.93,22.50,0.0,0.0,0.0,0.0,6,0,0,0,17,1,40.495000,972.89,972.89,0,0,0.002118,0.373219,137.773213,0.000000,25080,37916,265,194,71,25642,12274,168,51,<NA>


In [10]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283442 entries, 0 to 283441
Data columns (total 52 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   customer_id                       283442 non-null  object 
 1   MONTH                             283442 non-null  object 
 2   YEAR                              283442 non-null  object 
 3   pago_final_0                      283442 non-null  float64
 4   dif_pago_final_prev_month         283442 non-null  float64
 5   dif_pago_final_prev_2_month       283442 non-null  float64
 6   dif_pago_final_prev_3_month       283442 non-null  float64
 7   periodica_0                       283442 non-null  float64
 8   dif_periodica_prev_month          283442 non-null  float64
 9   dif_periodica_prev_2_month        283442 non-null  float64
 10  dif_periodica_prev_3_month        283442 non-null  float64
 11  consumo_0                         283442 non-null  f

In [11]:
sample.columns.tolist()

['customer_id',
 'MONTH',
 'YEAR',
 'pago_final_0',
 'dif_pago_final_prev_month',
 'dif_pago_final_prev_2_month',
 'dif_pago_final_prev_3_month',
 'periodica_0',
 'dif_periodica_prev_month',
 'dif_periodica_prev_2_month',
 'dif_periodica_prev_3_month',
 'consumo_0',
 'dif_consumo_prev_month',
 'dif_consumo_prev_2_month',
 'dif_consumo_prev_3_month',
 'aperiodica_0',
 'dif_aperiodica_prev_month',
 'dif_aperiodica_prev_2_month',
 'dif_aperiodica_prev_3_month',
 'discount_0',
 'dif_discount_prev_month',
 'dif_discount_prev_2_month',
 'dif_discount_prev_3_month',
 'ajuste_0',
 'dif_ajuste_prev_month',
 'dif_ajuste_prev_2_month',
 'dif_ajuste_prev_3_month',
 'Tota_Compra_disp',
 'Curr_Compra_disp',
 'Curr_Compra_Finanz_disp',
 'Curr_Finanz_disp',
 'Month_purchase_disp',
 'Modelo_disp',
 'Import_Rest_quota_disp',
 'pvp_total_disp',
 'pvp_total_disp_movil',
 'Curr_cancel_disp',
 'NUM_GB_OWNN_CURR',
 'NUM_GB_2G_CURR',
 'NUM_GB_3G_CURR',
 'NUM_GB_4G_CURR',
 'NUM_GB_5G_CURR',
 'NUM_SESS_CURR',
 

In [12]:
def assess_NA(data: pd.DataFrame):
    """
    Returns a pd.DataFrame denoting the total number of NA
    values and the percentage of NA values in each column.
    """
    # pd.Datadenoting features and the sum of their null values
    nulls = data.isnull().sum().reset_index().rename(columns={0: "count"})
    nulls["percent"] = nulls["count"] * 100 / len(data)

    return nulls

nulls = assess_NA(sample)

In [13]:
nulls.head(20)

,index,count,percent
0,customer_id,0,0.0
1,MONTH,0,0.0
2,YEAR,0,0.0
3,pago_final_0,0,0.0
4,dif_pago_final_prev_month,0,0.0
5,dif_pago_final_prev_2_month,0,0.0
6,dif_pago_final_prev_3_month,0,0.0
7,periodica_0,0,0.0
8,dif_periodica_prev_month,0,0.0
9,dif_periodica_prev_2_month,0,0.0


In [14]:
na_info_sorted = nulls.sort_values(by='percent', ascending=False)
na_info_sorted.head(20)


,index,count,percent
51,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA,266102,93.882346
33,Import_Rest_quota_disp,225676,79.619816
31,Month_purchase_disp,183121,64.606163
32,Modelo_disp,183121,64.606163
35,pvp_total_disp_movil,183121,64.606163
34,pvp_total_disp,179070,63.176946
37,NUM_GB_OWNN_CURR,175367,61.870506
28,Curr_Compra_disp,175367,61.870506
29,Curr_Compra_Finanz_disp,175367,61.870506
30,Curr_Finanz_disp,175367,61.870506


In [11]:
sample['customer_id'].nunique()

26312

* 26k unique users in this subsample
* 7% of positive class

### Prepare data for first model

1. First selection of cols (let's include this in ``data_cleaning()``)
2. Clean, normalize, drop, etc (idem)
3. Create feature target (`feature_computation()`)
4. Agreggate data if needed to reflect the past user behavior (idem)


#### 1. First selection

We said we will use for the moment the current precooked metrics for payments, discounts, and consumpitons, which are given by difference month on month.

In [109]:
# user-info cols to aggregate data later on
users_cols = ['customer_id', 'MONTH', 'YEAR']

# pre-cooked features
diff_cols = ['dif_pago_final_prev_month', 
                   'dif_pago_final_prev_2_month', 
                   'dif_pago_final_prev_3_month', 
                   'dif_consumo_prev_month', 
                   'dif_consumo_prev_2_month', 
                   'dif_consumo_prev_3_month', 
                   'dif_discount_prev_month', 
                   'dif_discount_prev_2_month', 
                   'dif_discount_prev_3_month']

# to-be-cooked features
transform_cols = ['pago_final_0']

# target
target_col = ['NUM_DAYS_LINE_TYPE_FIXE_POST_DEA']

In [88]:
def get_initial_params():
    """
    Loads all relevant parameters into a global variable. These parameters are then accessible to other functions in the script.
    Each function can consume the parameters it requires for its operation.
    This approach ensures centralized management and consistency of parameters across different functions.

    Returns:
        None: This function does not return a value but populates a global variable
        with necessary parameters.
    """

    config = configparser.ConfigParser()
    config.read("src/params.ini")
    global train_from, train_to
    train_from = config.get("PARAMS", "train_from")
    train_to = config.get("PARAMS", "train_to")

In [89]:
def data_cleaning(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans raw data by handling missing values, removing duplicates, correcting errors, and performing type conversions for data quality and consistency.
    Returns:
        DataFrame: Pandas DataFrame with cleaned and preprocessed data.
    """
    logger.info("Starting cleaning data")

    filter_df = raw_df[users_cols + diff_cols + monthly_cols + target_col]
    clean_df = filter_df.dropna(how="all")

    logger.info("Completed cleaning data!")
    return clean_df

In [90]:
def feature_computation(clean_data: pd.DataFrame, train_from : str = train_from, train_to : str = train_to) ->
    (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    """
    Split data into train and test features set, aggregate the data into historical behavior for those cols needed.
    It also joins it with already calculated features, and extract the needed target from 2 months ahead.
    Args:
        clean_data: The cleaned dataset with customer, month, and payment information.
        train_from: The starting date of the training period.
        train_to: The ending date of the training period.
    
    Returns:
        DataFrame: Pandas DataFrame with computed features for model training.
        DataFrame: Pandas DataFrame representing the target variable for train set.
        DataFrame: Pandas DataFrame with computed features for model testing.
        DataFrame: Pandas DataFrame representing the target variable for test set.
    """
    logger.info("Starting feature computation")

    # Convert the train_from and train_to to datetime
    train_from_dt = pd.to_datetime(train_from)
    train_to_dt = pd.to_datetime(train_to)

    # Filter train and test data before feature computation
    target_train_month = train_to + pd.DateOffset(months=2)
    test_from = train_from + pd.DateOffset(months=2)
    test_to = train_to + pd.DateOffset(months=2)
    target_test_month = test_to + pd.DateOffset(months=2)

    # Filter data for the combined range of training and testing period
    # Create date col to mix month and year
    clean_data['date'] = pd.to_datetime(clean_data['YEAR'].astype(str) + '-' + clean_data['MONTH'].astype(str) + '-01')
    combined_data = clean_data[(clean_data['DATE'] >= train_from_dt) & (clean_data['DATE'] <= test_to_dt)]
    combined_data.sort_values(by=['customer_id', 'DATE'], inplace=True)

    # Perform feature computations for the combined dataset
    combined_data['pago_final_prev_month'] = combined_data.groupby('customer_id')['pago_final_0'].shift(1)
    combined_data['pago_final_avg_3_months'] = combined_data.groupby('customer_id')['pago_final_0'].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )

    # Split the combined dataset into training and testing sets
    final_features_train = combined_data[combined_data['DATE'] <= train_to_dt]
    final_features_test = combined_data[(combined_data['DATE'] >= test_from_dt)]

    # Aggregate the most recent month's data per customer
    aggregated_train_data = final_features_train.groupby('customer_id').tail(1)
    aggregated_test_data = final_features_test.groupby('customer_id').tail(1)

    # Join the aggregated data with diff_cols from clean_data
    features_train = pd.merge(aggregated_train_data, clean_data[diff_cols], on='customer_id', how='left')
    features_test = pd.merge(aggregated_test_data, clean_data[diff_cols], on='customer_id', how='left')


    # Extract the target for the training and testing sets
    target_train = clean_data[clean_data['DATE'] == target_train_month][['customer_id', target_col]].set_index('customer_id')
    target_test = clean_data[clean_data['DATE'] == target_test_month][['customer_id', target_col]].set_index('customer_id')

    # Replace NaN values with -1 and convert targets to binary
    for target in [target_train, target_test]:
        target[target_col].fillna(-1, inplace=True)
        target['target'] = np.where(target[target_col] >= 0, 1, 0)


    logger.info(f"Features: {features.columns.tolist()}")
    logger.info(f"Target: {target.columns.tolist()}")
    logger.info("Completed feature computation!")

    return final_features_train, target_train, final_features_test, target_test


In [91]:
get_initial_params()
logger.info(f"Train from {train_from} to {train_to}")
clean_data = data_cleaning(sample)
#features, target = feature_computation(clean_data)

INFO - Train from 01-01-2022 to 31-12-2023
INFO - Starting cleaning data
INFO - Completed cleaning data!


In [92]:
clean_data.head()

,customer_id,MONTH,YEAR,dif_pago_final_prev_month,dif_pago_final_prev_2_month,dif_pago_final_prev_3_month,dif_consumo_prev_month,dif_consumo_prev_2_month,dif_consumo_prev_3_month,dif_discount_prev_month,dif_discount_prev_2_month,dif_discount_prev_3_month,pago_final_0,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA
0,4142337,12,2022,-0.92,5.00,13.15,-83.33,109.00,-288.48,82.40,-104.83,299.39,87.1499,<NA>
1,620962,10,2022,22.35,4.19,15.41,-165.29,-60.96,-95.95,163.79,67.89,117.37,88.0450,<NA>
2,127593,01,2022,0.89,22.48,2.95,33.88,29.52,7.54,-32.99,-4.96,0.50,177.5206,<NA>
3,1464466,07,2022,7.58,7.51,11.08,-39.25,-67.05,-155.04,49.13,76.86,167.81,106.6281,<NA>
4,311788,11,2022,-0.60,-1.50,3.00,-29.83,-58.43,-20.02,29.23,56.93,22.50,159.0900,<NA>


In [100]:
# clean_data['MONTH'] = pd.to_datetime(clean_data['MONTH'], errors='coerce')
clean_data.sort_values(by=['customer_id', 'MONTH'], inplace=True)
# Group by user and shift `pago_final_0` to create a 'pago_final_prev_month' feature
clean_data['pago_final_prev_month'] = clean_data.groupby('customer_id')['pago_final_0'].shift(1)

# Compute the average of `pago_final_0` for the last 3 months using rolling windows
# Note that the rolling function needs to be applied on the groupby object for each customer
clean_data['pago_final_avg_3_months'] = clean_data.groupby('customer_id')['pago_final_0']\
    .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
clean_data.head()[['customer_id', 'MONTH', 'pago_final_0', 'pago_final_prev_month', 'pago_final_avg_3_months']]

,customer_id,MONTH,pago_final_0,pago_final_prev_month,pago_final_avg_3_months
62451,100004,01,52.4526,NaN,52.4526
68227,100004,02,52.0450,52.4526,52.2488
11731,100004,03,52.7366,52.0450,52.4114
71667,100004,04,53.6595,52.7366,52.8137
158144,100004,05,52.1176,53.6595,52.8379


In [101]:
aggregated_data = clean_data.groupby('customer_id').tail(1)
aggregated_data.head()

,customer_id,MONTH,YEAR,dif_pago_final_prev_month,dif_pago_final_prev_2_month,dif_pago_final_prev_3_month,dif_consumo_prev_month,dif_consumo_prev_2_month,dif_consumo_prev_3_month,dif_discount_prev_month,dif_discount_prev_2_month,dif_discount_prev_3_month,pago_final_0,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA,pago_final_prev_month,pago_final_avg_3_months
11659,100004,12,2022,-5.02,1.52,1.44,-3.92,-49.01,21.57,-1.09,203.52,-51.58,59.1451,<NA>,64.1617,60.311733
214383,1000699,12,2022,0.90,0.90,0.60,-60.75,-157.72,-31.78,61.65,158.62,32.38,59.8001,<NA>,58.9001,59.200100
183988,100294,12,2022,31.56,58.20,58.20,350.68,834.91,834.91,-224.90,-1051.42,-1051.42,58.1998,<NA>,26.6400,42.419900
224879,100364,12,2022,0.60,0.60,0.30,-92.88,19.60,3.91,93.48,-19.00,-3.61,53.1451,<NA>,52.5451,52.745100
208518,100551,12,2022,0.30,0.30,-0.30,10.66,1.13,-18.63,-10.36,-0.83,18.33,68.2901,<NA>,67.9901,68.090100


In [102]:
type(train_to)

str